# Mega Sena - Sugestão via Moda e Cesta de Compras (affinity)

Implementação de sugestão de jogo de mega sena com objetivo de treinamento em técnicas de affinity e Moda. 
Fote de dados obitida em: javascript:document.frm51s.submit()

### Importando pacotes necessários e carregando os dados:

In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('mega_sena.xlsx', skiprows=6)

In [2]:
df.describe()

,Concurso,bola 1,bola 2,bola 3,bola 4,bola 5,bola 6
count,2065.000000,2067.000000,2068.000000,2068.000000,2067.000000,2067.000000,2067.000000
mean,1033.000000,30.532430,29.991292,30.990809,30.815100,30.525166,30.124397
std,596.258473,17.323335,17.420742,17.410450,17.252543,17.228396,17.269475
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,517.000000,15.000000,15.000000,16.000000,16.000000,15.000000,15.000000
50%,1033.000000,31.000000,30.000000,31.000000,31.000000,31.000000,30.000000
75%,1549.000000,46.000000,45.000000,46.000000,45.000000,45.000000,45.000000
max,2065.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000


### Ajustando os dados para que cada bola de cada jogo represente um registro

In [3]:
for ball in np.arange(1, 7):
    n = np.full((len(df), 1), ball)
    loteryball = df[['Concurso', 'Data', ('bola ' + str(ball))]].join(pd.DataFrame(n))
    loteryball.columns = ['Concurso', 'Data', 'Bola', 'Ord']
    if ball == 1:
        lotery = loteryball
    else: lotery = lotery.append(loteryball, ignore_index=True)

### Checando e removendo inconsistencias:

In [4]:
len(lotery[lotery.Bola < 0])

0

In [6]:
lotery = lotery[lotery.Bola > 0]

### Identificando os números mais sortiados:

Os números mais sortiados servem de sugestão para jogos com mais de 6 númeoros.

In [4]:
primeiro = list(lotery.Bola.mode())
print("First most comumn lotery numbers: {0}".format(primeiro))
segundo = list(lotery[lotery.Bola != 53]['Bola'].mode())
print("Second most comumn lotery numbers: {0}".format(segundo))
terceiro = lotery[lotery.Bola != 53][lotery.Bola != 5][lotery.Bola != 10]['Bola'].mode().tolist()
print("Second most comumn lotery numbers: {0}".format(terceiro))
most = primeiro + segundo + terceiro

First most comumn lotery numbers: [53.0]
Second most comumn lotery numbers: [5.0, 10.0]
Second most comumn lotery numbers: [23.0]


C:\Users\marce\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


## Início do algoritimo de identificação de números sorteados por Afinidade:

In [5]:
# Get list of unique items
balls = np.arange(1, 61) # for purchase axemples: itemList=list(set(userItemData["ItemId"].tolist()))

#Get count of itens
NumsCount = 60 # for purchase axemples: len(set(userItemData["ItemId"].tolist()))

### Identificação dos pares de sorteio de maior afinidade:

In [6]:
#Create an empty data frame to store ball affinity scores between two balss.
ballAffinity= pd.DataFrame(columns=('ball1', 'ball2', 'score'))
rowCount=0

In [7]:
# For each ball in the list, compare with other balls.
for ind1 in range(len(balls)):
    
    #Get list of games who lottery this ball 1.
    ball1 = lotery[lotery['Bola']==balls[ind1]]["Concurso"].tolist()
    
    #Get Ball 2 - balls that are not ball 1 or those that are not analyzed already.
    for ind2 in range(ind1, len(balls)):
        
        if ( ind1 == ind2):
            continue
       
        # Get list of games who lottery ball 2
        ball2=lotery[lotery['Bola']==balls[ind2]]["Concurso"].tolist()
        
        # Find score. 
        commonBalls= len(set(ball1).intersection(set(ball2)))
        score = commonBalls / NumsCount

        # Add a score for balls 1 and 2
        ballAffinity.loc[rowCount] = [balls[ind1],balls[ind2],score]
        rowCount +=1
        '''
        # Not necessery for lotery case
        # Add a score for 2 and 1. The same score would apply irrespective of the sequence.
        ballAffinity.loc[rowCount] = [balls[ind2],balls[ind1],score]
        rowCount +=1
        '''

In [8]:
#Check final result
paresMaisComuns = ballAffinity[ballAffinity.score > 0.50]
paresMaisComuns.sort_values(by='score',ascending=False)

,ball1,ball2,score
257,5.0,33.0,0.533333
517,10.0,33.0,0.516667


In [9]:
duplas = set((paresMaisComuns.ball1.unique().tolist()) + (paresMaisComuns.ball2.unique().tolist()))

In [10]:
print("Pares de loeterias mais comuns:", sorted(duplas))

Pares de loeterias mais comuns: [5.0, 10.0, 33.0]


Observe que os dois primeiros numeros corresponde a dois números com segunda maior modas (mais sorteados), embora eles não apresente um elevevado score como duplas entre si.

### Identificação dos trios de sorteio de maior afinidade:
Atensão: Este processo leva uns 3 minutos dependendo de sua máquina!

In [11]:
#Create an empty data frame to store ball affinity scores between 3 balss.
ballAffinity= pd.DataFrame(columns=('ball1', 'ball2', 'ball3', 'score'))
rowCount=0

#For each ball in the list, compare with other balls.
for ind1 in range(len(balls)):
    
    #Get list of games who lottery this ball 1.
    ball1 = lotery[lotery['Bola']==balls[ind1]]["Concurso"].tolist()
    
    #Get Ball 2 - balls that are not ball 1 or those that are not analyzed already.
    for ind2 in range(ind1, len(balls)):
        
        if ( ind1 == ind2):
            continue
       
        #Get list of games who lottery ball 2
        ball2=lotery[lotery['Bola']==balls[ind2]]["Concurso"].tolist()

        #Get Ball 3 - balls that are not ball 1 and 2 or those that are not analyzed already.
        for ind3 in range(ind2, len(balls)):
        
          if ( ind3 == ind2) or ( ind3 == ind1) :
              continue
       
          #Get list of games who lottery ball 3
          ball3=lotery[lotery['Bola']==balls[ind3]]["Concurso"].tolist()

          #Find score. 
          commonBalls = len(set(ball1).intersection(set(ball2)).intersection(set(ball3)))
          score= commonBalls / NumsCount

          #Add a score for balls 1, 2 and 3
          ballAffinity.loc[rowCount] = [balls[ind1],balls[ind2],balls[ind3],score]
          rowCount +=1

In [12]:
#Check final result
triosMaisComuns = ballAffinity[ballAffinity.score > 0.11]
triosMaisComuns.sort_values(by='score',ascending=False)

,ball1,ball2,ball3,score
29483,29.0,38.0,50.0,0.133333
2472,2.0,18.0,30.0,0.116667
6063,4.0,30.0,59.0,0.116667
18664,14.0,33.0,36.0,0.116667
21976,18.0,21.0,38.0,0.116667
24637,21.0,30.0,52.0,0.116667
26466,24.0,25.0,42.0,0.116667
29324,29.0,32.0,38.0,0.116667
29591,29.0,44.0,47.0,0.116667


In [18]:
trios = (set((triosMaisComuns.ball1.unique().tolist()) + 
            (triosMaisComuns.ball2.unique().tolist()) +
            (triosMaisComuns.ball3.unique().tolist())))
print("Trios de loterias mais comuns:",sorted(trios))

Trios de loterias mais comuns: [2.0, 4.0, 14.0, 18.0, 21.0, 24.0, 25.0, 29.0, 30.0, 32.0, 33.0, 36.0, 38.0, 42.0, 44.0, 47.0, 50.0, 52.0, 59.0]


Como existe um grande número de trios com mais de 11% de <b>afinidade</b>, vamos considerar apenas os trios acima de <b>12%</b>.

In [13]:
#Check final result
triosMaisComuns = ballAffinity[ballAffinity.score > 0.12]
triosMaisComuns.sort_values(by='score',ascending=False)
trios = (set((triosMaisComuns.ball1.unique().tolist()) + 
            (triosMaisComuns.ball2.unique().tolist()) +
            (triosMaisComuns.ball3.unique().tolist())))
print("Trios de loterias mais comuns:",sorted(trios))

Trios de loterias mais comuns: [29.0, 38.0, 50.0]


## Jogo Sugerido:

Com base nos números sorteados com maior afinade em pares e trios o jogo sugerido é:

In [14]:
Jogo = duplas.union(trios)
print("Jogo sugerido: ", sorted(Jogo))

Jogo sugerido:  [5.0, 10.0, 29.0, 33.0, 38.0, 50.0]


### Verificando se o jogo sugerido já foi sortiado alguma vez:

In [15]:
# Check if lottery game has been done
gamedone = False
for ball in np.arange(0, len(df)):
    game = set([df['bola 1'][ball], df['bola 2'][ball], df['bola 3'][ball], 
                df['bola 4'][ball], df['bola 5'][ball], df['bola 6'][ball]])
    if game == Jogo: 
        print("Já saiu no concurso", df['Concurso'][ball], df['Data'][ball])
        gamedone = True
if not(gamedone): print("O jogo sugerido ainda não foi sorteado!")

O jogo sugerido ainda não foi sorteado!


## Sugetão de jogo com mais de 6 números:

In [16]:
MaiorJogo = sorted(Jogo.union(set(most)))
print("Jogo com", len(MaiorJogo), "números: ", MaiorJogo)

Jogo com 8 números:  [5.0, 10.0, 23.0, 29.0, 33.0, 38.0, 50.0, 53.0]


O números adicionais correspondem são [23 e 53], respectivos ao terceiro e primeiro números mais sorteados.